Dans ce billet, nous allons [comme il y a quelques semaines](http://flothesof.github.io/podcast-rendez-vous-avec-X-fr.html), construire un flux RSS pour mon émission radio préférée : *Sur les épaules de Darwin*, animée par l'extraordinaire Jean Claude Ameisen.

Afin de faire ceci, nous allons partir du [travail de Clément Grimal](http://clementgrimal.fr/darwin/), qui maintient une liste à jour des émissions avec des liens de téléchargement. Une question, avant de commencer. Pourquoi faire ce travail, alors que le site de Clément permet déjà d'écouter à volonté ces émissions ? Tout simplement parce que son site ne permet pas de faire une recherche plein texte sur le contenu des émissions et également parce que j'aime écouter les émissions dans un logiciel de podcast afin de marquer ma progression.

# Récupération des liens vers les émissions

In [1]:
base_url = "http://clementgrimal.fr/darwin/"

In [2]:
import requests

In [3]:
r = requests.get(base_url)
r.encoding = 'utf-8'

On utilise beautiful soup pour déconstruire le résultat et chercher les différents liens dont nous avons besoin :

- titre de l'émission
- date de diffusion
- lien de téléchargement
- lien vers la page du site France Inter (pour en extraire la description de l'émission)

In [4]:
from bs4 import BeautifulSoup

In [5]:
soup = BeautifulSoup(r.text, 'html.parser')

On peut extraire toutes les émissions comme ceci :

In [6]:
all_shows = ([item for item in (li.find('table') for li in soup.findAll('li')) if item])

In [7]:
len(all_shows)

365

A partir de la première émission, on peut obtenir le lien vers le mp3 ainsi que le site France Inter, de même que le titre et la date de diffusion :

In [8]:
show = all_shows[0]

In [9]:
show.find('a', class_='download-link').attrs['href'].strip()

'http://prevost.pascal.free.fr/public/podcast/sur_les_epaules_de_darwin/Jean-Claude%20Ameisen%20-%20SUR%20LES%20EPAULES%20DE%20DARWIN%2004.09.2010.mp3'

In [10]:
show.find('span').find('a').attrs['href']

'http://www.franceinter.fr/em/sur-les-epaules-de-darwin/94428'

In [11]:
show.find('span').find('a').text

"La théorie de l'évolution de Charles Darwin"

In [12]:
str(show.find('span').find('a').next_sibling)[14:]

'4 Septembre 2010'

Ecrivons maintenant une boucle sur tous les épisodes :

In [13]:
show_data = []
for show in all_shows:
    download_link = show.find('a', class_='download-link')
    description_link = show.find('span').find('a').attrs['href']
    title = show.find('span').find('a').text
    date = str(show.find('span').find('a').next_sibling)[14:]
    if download_link:
        if download_link.attrs['href'].strip().startswith('.'):
            download_link = "http://clementgrimal.fr/darwin" + download_link.attrs['href'].strip()[1:]
        else:
            download_link = download_link.attrs['href'].strip()
        show_data.append([download_link, description_link, title, date])

Construisons un tableau avec ces données :

In [14]:
import pandas as pd

In [15]:
df = pd.DataFrame(show_data, columns=['lien mp3', 'lien description', 'titre', 'date'])
df.head(10)

,lien mp3,lien description,titre,date
0,http://prevost.pascal.free.fr/public/podcast/s...,http://www.franceinter.fr/em/sur-les-epaules-d...,La théorie de l'évolution de Charles Darwin,4 Septembre 2010
1,http://prevost.pascal.free.fr/public/podcast/s...,http://www.franceinter.fr/em/sur-les-epaules-d...,Le propre de l'homme?,11 Septembre 2010
2,http://prevost.pascal.free.fr/public/podcast/s...,http://www.franceinter.fr/em/sur-les-epaules-d...,Nos mémoires,18 Septembre 2010
3,http://prevost.pascal.free.fr/public/podcast/s...,http://www.franceinter.fr/em/sur-les-epaules-d...,Nos émotions,25 Septembre 2010
4,http://prevost.pascal.free.fr/public/podcast/s...,http://www.franceinter.fr/em/sur-les-epaules-d...,"Longévité, jeunesse et vieillissement",2 Octobre 2010
5,http://prevost.pascal.free.fr/public/podcast/s...,http://www.franceinter.fr/em/sur-les-epaules-d...,MORT CELLULAIRE ET SCULPTURE DU VIVANT,9 Octobre 2010
6,http://prevost.pascal.free.fr/public/podcast/s...,http://www.franceinter.fr/em/sur-les-epaules-d...,NAISSANCES,16 Octobre 2010
7,http://prevost.pascal.free.fr/public/podcast/s...,http://www.franceinter.fr/em/sur-les-epaules-d...,Biodiversité,23 Octobre 2010
8,http://prevost.pascal.free.fr/public/podcast/s...,http://www.franceinter.fr/em/sur-les-epaules-d...,mort cellulaire et sculpture du vivant (2),30 Octobre 2010
9,http://prevost.pascal.free.fr/public/podcast/s...,http://www.franceinter.fr/em/sur-les-epaules-d...,Un voyage avec Oliver Sacks ( 1 ),6 Novembre 2010


In [16]:
df.tail(10)

,lien mp3,lien description,titre,date
351,http://clementgrimal.fr/darwin/files/2017-07-0...,https://www.franceinter.fr/emissions/sur-les-e...,Dans les abysses,1 Juillet 2017
352,http://clementgrimal.fr/darwin/files/2017-07-0...,https://www.franceinter.fr/emissions/sur-les-e...,Naissance d'un mythe,8 Juillet 2017
353,http://clementgrimal.fr/darwin/files/2017-07-1...,https://www.franceinter.fr/emissions/sur-les-e...,A la rencontre de la pensée chinoise (1),15 Juillet 2017
354,http://clementgrimal.fr/darwin/files/2017-07-2...,https://www.franceinter.fr/emissions/sur-les-e...,A la rencontre de la pensée chinoise (2),22 Juillet 2017
355,http://clementgrimal.fr/darwin/files/2017-07-2...,https://www.franceinter.fr/emissions/sur-les-e...,Déluges,29 Juillet 2017
356,http://clementgrimal.fr/darwin/files/2017-08-0...,https://www.franceinter.fr/emissions/sur-les-e...,Le passage du Nord-Ouest,5 Août 2017
357,http://clementgrimal.fr/darwin/files/2017-08-1...,https://www.franceinter.fr/emissions/sur-les-e...,La faille de Cascadia,12 Août 2017
358,http://clementgrimal.fr/darwin/files/2017-08-1...,https://www.franceinter.fr/emissions/sur-les-e...,Le mystérieux tsunami de 1700 au Japon,19 Août 2017
359,http://clementgrimal.fr/darwin/files/2017-08-2...,https://www.franceinter.fr/emissions/sur-les-e...,Des récits et des masques,26 Août 2017
360,http://clementgrimal.fr/darwin/files/2017-09-0...,https://www.franceinter.fr/emissions/sur-les-e...,Clotho et le fil de la vie,2 Septembre 2017


In [17]:
df.describe()

,lien mp3,lien description,titre,date
count,361,361,361,361
unique,361,361,289,361
top,http://clementgrimal.fr/darwin/files/2017-06-1...,https://www.franceinter.fr/emissions/sur-les-e...,Le souci de l'autre,4 Janvier 2014
freq,1,1,3,1


Nous voilà maintenant en possession des liens vers 361 épisodes publiés sur le site de Clément Grimal. On remarque que certaines d'entre elles sont des rediffusions (289 épisodes uniques sur 361). 

Téléchargeons maintenant les descriptions des émissions.

# Téléchargement des descriptions à partir du site de France Inter 

Nous avons les liens vers les descriptions : il suffit donc de lire la page de chaque émission et d'en extraire le contenu intéressant.

In [18]:
link = df['lien description'][0]
link

'http://www.franceinter.fr/em/sur-les-epaules-de-darwin/94428'

In [19]:
soup = BeautifulSoup(requests.get(link).text, 'html.parser')

In [20]:
article = soup.find('article')
article

<article class="content-body"><p><strong>Bibliographie:</strong></p><p><strong>L'origine des espèces de Charles Darwin aux Editions Flammarion Origines,</strong></p><div class="onf-ad" id="sas_28990"></div><script type="text/javascript">
    window._sasFormats = window._sasFormats || [];

    window._sasFormats.push(function() {
        window.sas.call('std', {
            siteId:70662,
            pageId:537605,
            formatId:28990,
            target: 'Culture'
        });
    });
</script><p><strong>Lettres choisies (1828-1859)de Charles Darwin traduit de l'anglais par Mickael Popelard préface de Stefen Jay Gould aux Editions Bayard</strong></p><p><strong>L'expression des émotions chez les hommes et les animaux de Charles Darwin aux Editions Rivages</strong></p><p><strong>La lettre volée, d'Edgar Allan Poe, Ed Folio Gallimard</strong></p><p><strong>La mal-mesure de l'homme de Stephen Jay Gould aux Editions Odile Jacob</strong></p><p><strong>Identité et violence: l'illusion du

In [21]:
article.find('script').extract()

<script type="text/javascript">
    window._sasFormats = window._sasFormats || [];

    window._sasFormats.push(function() {
        window.sas.call('std', {
            siteId:70662,
            pageId:537605,
            formatId:28990,
            target: 'Culture'
        });
    });
</script>

In [22]:
"\n".join(article.strings)

'Bibliographie:\nL\'origine des espèces de Charles Darwin aux Editions Flammarion Origines,\nLettres choisies (1828-1859)de Charles Darwin traduit de l\'anglais par Mickael Popelard préface de Stefen Jay Gould aux Editions Bayard\nL\'expression des émotions chez les hommes et les animaux de Charles Darwin aux Editions Rivages\nLa lettre volée, d\'Edgar Allan Poe, Ed Folio Gallimard\nLa mal-mesure de l\'homme de Stephen Jay Gould aux Editions Odile Jacob\nIdentité et violence: l\'illusion du destin d\'Amartya Sen aux editions Odile Jacob\n"Dans la Lumière et les Ombres. Darwin et le bouleversement du monde" de Jean-Claude Ameisen aux Editions Fayard / Seuil\ninvité(s)\nCharles Darwin 1809-1882\nprogrammation musicale\nBenjamin Biolay\nSi tu suis mon regard album: LA SUPERBElabel: NAIVE RECORDSparution: 2010\nJane Birkin\nApocalypstick album: EX FAN DES SIXTIESlabel: FONTANAparution: 1978\nLuke\nLE ROBOT album: LE ROBOTlabel: JIVE EPIC / SONYparution: 2010\nCat power\nTHE GREATEST album:

In [23]:
import tqdm

In [24]:
descriptions = []
for description_link in tqdm.tqdm(df['lien description']):
    soup = BeautifulSoup(requests.get(description_link).text, 'html.parser')
    article = soup.find('article')
    script = article.find('script')
    if script:
        script.extract()
    descriptions.append("\n".join(article.strings))

100%|██████████| 361/361 [01:48<00:00,  3.94it/s]


In [25]:
df['description'] = descriptions

In [26]:
df.head(10)

,lien mp3,lien description,titre,date,description
0,http://prevost.pascal.free.fr/public/podcast/s...,http://www.franceinter.fr/em/sur-les-epaules-d...,La théorie de l'évolution de Charles Darwin,4 Septembre 2010,Bibliographie:\nL'origine des espèces de Charl...
1,http://prevost.pascal.free.fr/public/podcast/s...,http://www.franceinter.fr/em/sur-les-epaules-d...,Le propre de l'homme?,11 Septembre 2010,programmation musicale\nSTAFF BENDA BILILI\nSa...
2,http://prevost.pascal.free.fr/public/podcast/s...,http://www.franceinter.fr/em/sur-les-epaules-d...,Nos mémoires,18 Septembre 2010,programmation musicale\nIggy POP\nKing of the ...
3,http://prevost.pascal.free.fr/public/podcast/s...,http://www.franceinter.fr/em/sur-les-epaules-d...,Nos émotions,25 Septembre 2010,programmation musicale\nEMILY LOISEAU\nL'autre...
4,http://prevost.pascal.free.fr/public/podcast/s...,http://www.franceinter.fr/em/sur-les-epaules-d...,"Longévité, jeunesse et vieillissement",2 Octobre 2010,programmation musicale\nBonga\nMona ki gni xiç...
5,http://prevost.pascal.free.fr/public/podcast/s...,http://www.franceinter.fr/em/sur-les-epaules-d...,MORT CELLULAIRE ET SCULPTURE DU VIVANT,9 Octobre 2010,programmation musicale\nNouvelle vague / Camil...
6,http://prevost.pascal.free.fr/public/podcast/s...,http://www.franceinter.fr/em/sur-les-epaules-d...,NAISSANCES,16 Octobre 2010,invité(s)\nRené Frydman\nChef du Pôle Mère-Enf...
7,http://prevost.pascal.free.fr/public/podcast/s...,http://www.franceinter.fr/em/sur-les-epaules-d...,Biodiversité,23 Octobre 2010,"""L'infinité des formes les plus belles et les ..."
8,http://prevost.pascal.free.fr/public/podcast/s...,http://www.franceinter.fr/em/sur-les-epaules-d...,mort cellulaire et sculpture du vivant (2),30 Octobre 2010,« Nous ne nous baignons pas deux fois dans le ...
9,http://prevost.pascal.free.fr/public/podcast/s...,http://www.franceinter.fr/em/sur-les-epaules-d...,Un voyage avec Oliver Sacks ( 1 ),6 Novembre 2010,A la rencontre de la singularité de nos mondes...


In [ ]:
df.tail(10)

,lien mp3,lien description,titre,date,description
351,http://clementgrimal.fr/darwin/files/2017-07-0...,https://www.franceinter.fr/emissions/sur-les-e...,Dans les abysses,1 Juillet 2017,\n Dans les abysses\n \nPlon...
352,http://clementgrimal.fr/darwin/files/2017-07-0...,https://www.franceinter.fr/emissions/sur-les-e...,Naissance d'un mythe,8 Juillet 2017,\n Naissance d'un mythe\n \n...
353,http://clementgrimal.fr/darwin/files/2017-07-1...,https://www.franceinter.fr/emissions/sur-les-e...,A la rencontre de la pensée chinoise (1),15 Juillet 2017,\n A la rencontre de la pensée chin...
354,http://clementgrimal.fr/darwin/files/2017-07-2...,https://www.franceinter.fr/emissions/sur-les-e...,A la rencontre de la pensée chinoise (2),22 Juillet 2017,\n A la rencontre de la pensée chin...
355,http://clementgrimal.fr/darwin/files/2017-07-2...,https://www.franceinter.fr/emissions/sur-les-e...,Déluges,29 Juillet 2017,\n Déluges\n \nCette émissio...
356,http://clementgrimal.fr/darwin/files/2017-08-0...,https://www.franceinter.fr/emissions/sur-les-e...,Le passage du Nord-Ouest,5 Août 2017,\n Le passage du Nord-Ouest\n ...
357,http://clementgrimal.fr/darwin/files/2017-08-1...,https://www.franceinter.fr/emissions/sur-les-e...,La faille de Cascadia,12 Août 2017,\n La faille de Cascadia\n \...
358,http://clementgrimal.fr/darwin/files/2017-08-1...,https://www.franceinter.fr/emissions/sur-les-e...,Le mystérieux tsunami de 1700 au Japon,19 Août 2017,\n Le mystérieux tsunami de 1700 au...
359,http://clementgrimal.fr/darwin/files/2017-08-2...,https://www.franceinter.fr/emissions/sur-les-e...,Des récits et des masques,26 Août 2017,\n Des récits et des masques\n ...
360,http://clementgrimal.fr/darwin/files/2017-09-0...,https://www.franceinter.fr/emissions/sur-les-e...,Clotho et le fil de la vie,2 Septembre 2017,\n Clotho et le fil de la vie.\n ...


# Taille des fichiers mp3 

Il nous manque encore la récupération des tailles de fichier mp3, qui sera inscrite dans le flux RSS :

In [ ]:
byte_lengths = []
for mp3_link in tqdm.tqdm(df['lien mp3']):
    r = requests.head(mp3_link)
    byte_lengths.append(r.headers['content-length'])

 66%|██████▌   | 238/361 [00:09<00:07, 16.34it/s]

In [ ]:
df['byte_length'] = byte_lengths

In [ ]:
df.head()

# Ecriture du fichier RSS 

Maintenant que nous avons collecté toutes les émissions, nous pouvons écrire un ficher RSS, comme nous l'avions déjà fait [dans le cas de l'émission Rendez vous avec X](http://flothesof.github.io/podcast-rendez-vous-avec-X-fr.html).

In [ ]:
import dateparser

In [ ]:
import xml.etree.cElementTree as ET

rss = ET.Element("rss", version="2.0")
channel = ET.SubElement(rss, "channel")
title = ET.SubElement(channel, "title")
title.text = 'Podcast Sur les épaules de Darwin'
description = ET.SubElement(channel, "description")
description.text = "Podcast inofficiel de l'émission Sur les épaules de Darwin, tiré du site http://clementgrimal.fr/darwin/ et https://www.franceinter.fr/emissions/sur-les-epaules-de-darwin"
for index, row in df.iterrows():
    item = ET.SubElement(channel, "item")
    item_title = ET.SubElement(item, "title")
    item_title.text = row['titre']
    item_description = ET.SubElement(item, "description")
    item_description.text = row['description']
    item_pubdate = ET.SubElement(item, "pubDate")
    item_pubdate.text = dateparser.parse(row.date).date().strftime('%a, %d %b %Y 11:00:00')
    item_enclosure = ET.SubElement(item, "enclosure", url='{}'.format(row['lien mp3']),
                                   length=row.byte_length,
                                  type="audio/mpeg")
tree = ET.ElementTree(rss)
tree.write("files/podcast_Sur_les_epaules_de_Darwin.xml", encoding='utf-8')